<h2>This notebook walks you through the second stage in SinkSAM framework, computation of closed depressions using Arcpy library.</h2>

In [ ]:
import os ##Fill-sinks
import arcpy

# Function to apply Fill and save filled rasters
def apply_fill_and_save(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            input_tiff = os.path.join(input_folder, filename)
            base_name = os.path.splitext(filename)[0]

            # Apply the Fill function with a Z limit of 2
            filled_raster = arcpy.sa.Fill(input_tiff)
            filled_raster_path = os.path.join(output_folder, f"{base_name}_filled.tif")
            filled_raster.save(filled_raster_path)

input_folder = "C:/Users/osher/Downloads/asaf_for_paper/RawCRS/"
output_folder = "C:/Users/osher/Downloads/asaf_for_paper/FilledRasters_2/"

apply_fill_and_save(input_folder, output_folder)

print("Step 1: Filled rasters saved.")


In [ ]:
import os ##Subtraction
import arcpy

# Function to subtract original georeferenced rasters from filled rasters and save results
def subtract_and_save(georef_folder, filled_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(georef_folder):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            base_name = os.path.splitext(filename)[0]
            georef_raster = os.path.join(georef_folder, filename)
            filled_raster = os.path.join(filled_folder, f"{base_name}_filled.tif")

            # Subtract the original georeferenced raster from the filled raster
            subtracted_raster = arcpy.sa.Minus(filled_raster, georef_raster)
            subtracted_raster_path = os.path.join(output_folder, f"{base_name}_subtracted.tif")
            subtracted_raster.save(subtracted_raster_path)

georef_folder = "C:/Users/osher/Downloads/asaf_for_paper/RawCRS/"
filled_folder = "C:/Users/osher/Downloads/asaf_for_paper/FilledRasters_2/"
output_folder = "C:/Users/osher/Downloads/asaf_for_paper/SubtractedRasters_2/"

subtract_and_save(georef_folder, filled_folder, output_folder)

print("Step 2: Subtracted rasters saved.")

In [ ]:
import rasterio ##Closed depression
from rasterio.merge import merge
import os

def combine_rasters(output_folder, combined_raster_path):
    # List to store the file paths of the subtracted rasters
    raster_files = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    # Open all rasters using rasterio
    src_files_to_mosaic = []
    for raster_file in raster_files:
        src = rasterio.open(raster_file)
        src_files_to_mosaic.append(src)
    
    # Combine rasters using the maximum value where they overlap
    mosaic, out_trans = merge(src_files_to_mosaic, method='max')
    
    # Save the combined raster
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans})
    
    with rasterio.open(combined_raster_path, "w", **out_meta) as dest:
        dest.write(mosaic)

output_folder = "C:/Users/osher/Downloads/asaf_for_paper/SubtractedRasters_2/"
combined_raster_path = "C:/Users/osher/Downloads/asaf_for_paper/CombinedRasters/combined_subtracted_raster.tif"

combine_rasters(output_folder, combined_raster_path)

print("Step 3: Combination of subtracted rasters completed.")

<h2> After this stage we pachiffy closed depression combined raster with RGB in ArcGIS Pro using "Export Training Data For Deep Learning (Image Analyst)"</h2>